In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import scipy
from numpy import mean
from matplotlib import pyplot as plt
!pip install antropy
import antropy as an
from scipy.stats import skew, kurtosis
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from keras import layers, models, regularizers
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import roc_curve,auc, precision_score,recall_score,f1_score

In [ ]:
Features_df = pd.read_csv('/content/extractedFeatures.csv')
Features_df.drop(Features_df.columns[0], axis = 1, inplace = True)      # In the csv file first column has number values from 0-2280 and carries redundadnt info.

#3 Data preparation
##3.1 Shuffling
Data = Features_df.sample(frac = 1)
features = Data[[x for x in Data.columns if x not in ["Label"]]]   # Data for training
Labels = Data['Label']                                            # Labels for training
Labels = Labels.astype('category')

In [ ]:
splitRatio = 0.3
train, test = train_test_split(Data ,test_size=splitRatio,
                               random_state = 123, shuffle = True)  # Spilt to training and testing data

train_X = train[[x for x in train.columns if x not in ["Label"]]]   # Data for training
train_Y = train['Label']                                            # Labels for training

###4.5.2 Testing Data
test_X = test[[x for x in test.columns if x not in ["Label"]]]     # Data fo testing
test_Y = test["Label"]                                              # Labels for training

###4.5.3 Validation Data
x_val = train_X[:200]                                                # 50 Sample for Validation
partial_x_train = train_X[200:]
partial_x_train = partial_x_train.values

y_val = train_Y[:200]
y_val = to_categorical(y_val)
partial_y_train = train_Y[200:]
partial_y_train = partial_y_train.values
partial_y_train = to_categorical(partial_y_train)

print("Data is prepeared")

print("Start Building Classifer")

#4 Classification Model

##4.1 Building Model

###4.1.1 Architecture
model = models.Sequential()
model.add(layers.Dense(200, activation = 'relu', input_shape=(17,),kernel_regularizer=regularizers.l1(0.01)))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(150, activation = 'relu'))
model.add(layers.Dense(100, activation = 'relu'))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(75, activation = 'relu'))
model.add(layers.Dense(48,  activation= 'sigmoid'))

####5.1.1.2 Hyper Parameters Tuning
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Classifier Bulit\n")
print("Start Training\n")

##5.1.2 Training Model
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs = 50,                # initial value = 1000
                    batch_size = 16,              # initial value = 16
                    validation_data=(x_val, y_val))

weights = model.get_weights()
configs = model.get_config()

# Featuers_weights = np.apply_along_axis(mean, 1, weights[0])



print("Finish Training")


#5 Model Evaluation

##5.1 Network Architecture
print(model.summary())

print("Start Evaluating Data")

##4.2 Training Process
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1,len(loss_values)+1)
plt.figure()
plt.plot(epochs, loss_values, 'bo', label="training loss", color='r')
plt.plot(epochs, val_loss_values, 'b', label="validation loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoches")
plt.ylabel("loss")
plt.legend()
plt.show()

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
plt.figure()
plt.plot(epochs, acc_values, 'bo', label="training acc", color = 'r')
plt.plot(epochs, val_acc_values, 'b', label="validation acc")
plt.title("Training and Validation acc")
plt.xlabel("Epoches")
plt.ylabel("acc")
plt.legend()
plt.show()

##5.3 Prediction
ANN_predictions = model.predict(test_X)

Pred = np.zeros([len(ANN_predictions)])
for i in range(0,len(ANN_predictions)):
    Pred[i] = list(ANN_predictions[i]).index(max(ANN_predictions[i]))
ANN_Pred = pd.Series(Pred)

####5.1.2.4 Metrics
print("Accuracy:",accuracy_score(test_Y, ANN_Pred))
print("f1 score:", f1_score(test_Y, ANN_Pred,average="micro"))
print("precision score:", precision_score(test_Y, ANN_Pred,average="micro"))
print("recall score:", recall_score(test_Y, ANN_Pred,average="micro"))
print("confusion matrix:\n",confusion_matrix(test_Y, ANN_Pred))
print("classification report:\n", classification_report(test_Y, ANN_Pred))

In [ ]:
print(test_X)

In [ ]:
# Import XGBoost library
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Define and train an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=48, random_state=123)
xgb_classifier.fit(train_X, train_Y)

# Make predictions on the testing data
xgb_predictions = xgb_classifier.predict(test_X)

# Evaluate the XGBoost model
accuracy = accuracy_score(test_Y, xgb_predictions)
f1 = f1_score(test_Y, xgb_predictions, average="micro")
precision = precision_score(test_Y, xgb_predictions, average="micro")
recall = recall_score(test_Y, xgb_predictions, average="micro")
conf_matrix = confusion_matrix(test_Y, xgb_predictions)
classification_rep = classification_report(test_Y, xgb_predictions)

# Print the evaluation metrics
print("XGBoost Model Evaluation:")
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

Data1 = pd.read_csv('/content/extractedFeatures.csv')
X= Data1[[x for x in Data.columns if x not in ["Label"]]]
Y=Data1['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

# Define and train multiple classifiers
classifiers = {
    'Random Forest': RandomForestClassifier(random_state=123),
    'Gradient Boosting': GradientBoostingClassifier(random_state=123),
    'SVM': SVC(random_state=123),
    # Add more classifiers as needed
}

results = {}

for name, classifier in classifiers.items():
    classifier.fit(X_train, Y_train)
    predictions = classifier.predict(X_test)

    accuracy = accuracy_score(Y_test, predictions)
    f1 = f1_score(Y_test, predictions, average='micro')
    precision = precision_score(Y_test, predictions, average='micro')
    recall = recall_score(Y_test, predictions, average='micro')
    conf_matrix = confusion_matrix(Y_test, predictions)
    classification_rep = classification_report(Y_test, predictions)

    results[name] = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'Confusion Matrix': conf_matrix,
        'Classification Report': classification_rep
    }

# Print the results for each classifier
for name, metrics in results.items():
    print(f'{name} Metrics:')
    print(f'Accuracy: {metrics["Accuracy"]}')
    print(f'F1 Score: {metrics["F1 Score"]}')
    print(f'Precision: {metrics["Precision"]}')
    print(f'Recall: {metrics["Recall"]}')
    print(f'Confusion Matrix:\n{metrics["Confusion Matrix"]}')
    print(f'Classification Report:\n{metrics["Classification Report"]}')
    print('\n')

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Notice:
[1] This code is only for reference.
Please modify the codes to fit your own data.
[2] The Code is based on TensorFlow 2.X.
Please install the TensorFlow 2.X version.

"""

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

# Read Training Data
train_data = train_X
train_data = np.array(train_data).astype('float32')
# Read Training Labels
train_labels = train_Y
train_labels = np.array(train_labels).astype('float32')
train_labels = tf.one_hot(indices=train_labels, depth=2)
train_labels = np.squeeze(train_labels)

# Read Testing Data
test_data = test_X
test_data = np.array(test_data).astype('float32')

# Read Testing Labels
test_labels = test_Y
test_labels = np.array(test_labels).astype('float32')
test_labels = tf.one_hot(indices=test_labels, depth=2)
test_labels = np.squeeze(test_labels)




class CatgoricalTP(tf.keras.metrics.Metric):
    def __init__(self, name='categorical_tp', **kwargs):
        super(CatgoricalTP, self).__init__(name=name, **kwargs)
        self.tp = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        y_true = tf.argmax(y_true, axis=-1)
        values = tf.equal(tf.cast(y_pred, 'int32'), tf.cast(y_true, 'int32'))
        values = tf.cast(values, 'float32')
        if sample_weight is not None:
            sample_weights = tf.cast(sample_weight, 'float32')
            values = tf.multiply(values, sample_weights)

        self.tp.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.tp

    def reset_states(self):
        self.tp.assign(0.)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.5):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim), ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out = self.layernorm2(out1 + ffn_output)

        return out


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen
        self.embed_dim = embed_dim

    def call(self, x):
        positions = tf.range(start=0, limit=self.maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = tf.reshape(x, [-1, self.maxlen, self.embed_dim])
        out = x + positions

        return out


maxlen = 17  # (Maximum) length of the signals
embed_dim = 1  # Number of features of one time point
num_heads = 8  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer


def get_model():
    # Input Time-series
    inputs = layers.Input(shape=(maxlen * embed_dim,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, embed_dim)
    x = embedding_layer(inputs)

    # Encoder Architecture
    transformer_block_1 = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)
    transformer_block_2 = TransformerBlock(embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim)
    x = transformer_block_1(x)
    x = transformer_block_2(x)

    # Output
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model


model = get_model()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy", CatgoricalTP()])

history = model.fit(
    train_data, train_labels, batch_size=64, epochs=100, validation_data=(test_data, test_labels)
)

model.save_weights('model_weight')

Epoch 1/100
31/32 [============================>.] - ETA: 0s - loss: 0.0314 - accuracy: 0.2389 - categorical_tp: 474.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2699: UserWarning: Metric CatgoricalTP implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


32/32 [==============================] - 7s 38ms/step - loss: 0.0313 - accuracy: 0.2351 - categorical_tp: 474.0000 - val_loss: 0.0233 - val_accuracy: 0.0139 - val_categorical_tp: 12.0000
Epoch 2/100
32/32 [==============================] - 1s 32ms/step - loss: 0.0313 - accuracy: 0.0238 - categorical_tp: 48.0000 - val_loss: 0.0233 - val_accuracy: 0.0139 - val_categorical_tp: 12.0000
Epoch 3/100
32/32 [==============================] - 1s 42ms/step - loss: 0.0313 - accuracy: 0.0238 - categorical_tp: 48.0000 - val_loss: 0.0233 - val_accuracy: 0.0139 - val_categorical_tp: 12.0000
Epoch 4/100
32/32 [==============================] - 1s 40ms/step - loss: 0.0313 - accuracy: 0.0238 - categorical_tp: 48.0000 - val_loss: 0.0233 - val_accuracy: 0.0139 - val_categorical_tp: 12.0000
Epoch 5/100
32/32 [==============================] - 1s 26ms/step - loss: 0.0313 - accuracy: 0.0238 - categorical_tp: 48.0000 - val_loss: 0.0233 - val_accuracy: 0.0139 - val_categorical_tp: 12.0000
Epoch 6/100
32/32 [==